<a href="https://colab.research.google.com/github/avy666/LLM_Sber/blob/main/home_task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Решение задачи Question Answering

**Цель домашнего задания**: решить с помощью bert-подобной модели задачу question answering (QA).

В теме №2 мы разобрали две самых популярных архитектуры на базе трансформера: ***BERT и GPT***. Изучили их основные особенности и дальнейшее развитие. Давайте научимся теперь дообучать подобные модели для различных задач. Например, для задачи поиска ответа на вопрос. В целом, любая хорошо обученная gpt-подобная модель с этим справится, но мы хотим обучить bert-подобную модель, чтобы убедиться, что для решения подобной задачи достаточно только энкодеров.

### Подготовка данных

В качестве академического бенчмарка для задачи QA чаще всего используется датасет SQuAD, состоящем из вопросов, заданных краудворкерами по набору статей Википедии, поэтому мы будем использовать именно его.

In [ ]:
!pip install transformers datasets -U -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Мы можем взглянуть на этот объект, чтобы узнать больше о датасете SQuAD:

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

Все необходимое содержится в полях context, question и answers, так что давайте выведем их для первого элемента нашего обучающего набора:

In [ ]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Question:  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer:  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


### Загрузка токенизатора

В качестве baseline будем использовать легкий дистилированный берт.

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert/distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Прверим работу нашего токенизатора:

In [ ]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ), is a simple, modern stone statue of Mary. [SEP]'

Нам надо быть готовым к тому, что будут встречаться длинные контексты, а значит, надо будет ввести предварительную обработку.

Чтобы увидеть, как это работает на текущем примере, мы можем ограничить длину до 100 и использовать скользящее окно из 50 токенов. Основные параметры предобработки следующие:

- ***max_length*** для установки максимальной длины (здесь 100)
- ***truncation="only_second"*** для усечения контекста (который находится во второй позиции), когда вопрос с его контекстом слишком длинный
- ***stride*** для задания количества перекрывающихся токенов между двумя последовательными фрагментами (здесь 50)
- ***return_overflowing_tokens=True***, чтобы сообщить токенизатору, что нам нужны токены переполнения (overflowing tokens)

In [ ]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basi [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Next to the Main Building is the B

Как мы можем видеть, наш пример был разбит на четыре части, каждый из которых содержит вопрос и часть контекста. Обратите внимание, что ответ на вопрос (“Bernadette Soubirous”) появляется только в третьей и последней части, поэтому, работая с длинными контекстами таким образом, мы невольно создадим несколько обучающих примеров, в которых ответ не будет включен в контекст. Для этих примеров метками будут start_position = end_position = 0 (таким образом мы предсказываем токен [CLS]). Мы также зададим эти метки в неудачном случае, когда ответ был усечен, то есть у нас будут только его начало (или конец). Для примеров, где ответ полностью находится в контексте, метками будут индекс токена, с которого начинается ответ, и индекс токена, на котором ответ заканчивается.

Датасет предоставляет нам начальный символ ответа в контексте, а прибавив к нему длину ответа, мы можем найти конечный символ в контексте. Чтобы сопоставить их с индексами токенов, нам нужно использовать сопоставление смещений, поэтому добавим в токенизатор еще ***return_offsets_mapping=True***:

In [ ]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

### Подготовка датасета

Теперь, когда мы шаг за шагом разобрались с предварительной обработкой обучающих данных, мы можем сгруппировать их в функцию, которую будем применять ко всему датасету.

In [ ]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Найдём начало и конец контекста
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # Если ответ не полностью находится внутри контекста, меткой будет (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # В противном случае это начальная и конечная позиции токенов
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

Обратите внимание, что мы ввели две константы для определения максимальной длины и длины скользящего окна, а также добавили немного очистки перед токенизацией: некоторые вопросы в датасете SQuAD имеют лишние пробелы в начале и конце, которые ничего не добавляют (и занимают место при токенизации, если вы используете модель вроде RoBERTa), поэтому мы удалили эти лишние пробелы.

Чтобы применить эту функцию ко всему обучающему набору, мы используем метод Dataset.map() с флагом batched=True.

In [ ]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

### Обработка валидационных данных

Совершенно аналогично выполним предобработку валидационных данных.

In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names
)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

### Дообучение модели

Кажется, у нас все готово для дообучения. Осталось только определить метрику качества. Нам нужно правильно угадать два токена (токен начала ответа и токен конца ответа). Помимо этого, надо чтобы все символы внутри ответа как можно ближе соответствовали символам в ответе. Для этого придумали две метрики: ***F1*** и ***exact match***.

F1 метрика для текста вычисляется по отдельным словам в прогнозе по сравнению со словами в истинном ответе. Количество общих слов между прогнозом и истинным ответом является основой оценки F1: точность — это отношение количества общих слов к общему количеству слов в прогнозе , а полнота — это отношение количества общих слов к общему количеству слов в истинном ответе.

$$ F_{1}=2\dfrac{precision*recall}{precision+recall}$$

Exact match более жесткая метрика, так как устроена она по принципу "все или ничего". Для того, чтобы она была равна 1 для одного примера необходимо полное совпадение всех символов в ответе, иначе она равна сразу 0.

Давайте реализуем теперь эти две метрики. Для начала нам надо нормализовать текст, поскольку в зависимости от модели, могут генерироваться разные посторонние символы и сам ответ может быть как с большой, так и с маленькой буквы. Чтобы избежать всего этого и проводят простую нормализацию.

In [ ]:
def normalize_text(s):
    """
    Функция для нормализации текста. Удаляет артикли, пунктуацию и приводит текст к единому виду,
    устраняя лишние пробелы и приводя все символы к нижнему регистру.

    Args:
    s (str): Входной текст.

    Returns:
    str: Нормализованный текст.
    """
    # Ваш код здесь

In [ ]:
def compute_exact_match(prediction, truth):
    """
    Функция для вычисления точного совпадения между предсказанием и истинным значением.

    Args:
    prediction (str): Предсказанный текст.
    truth (str): Истинный текст.

    Returns:
    int: 1, если нормализованный предсказанный текст совпадает с нормализованным истинным текстом, иначе 0.
    """
    # Ваш код здесь


In [ ]:
def compute_f1(prediction, truth):
    """
    Функция для вычисления F1-меры между предсказанным текстом и истинным текстом.

    Args:
    prediction (str): Предсказанный текст.
    truth (str): Истинный текст.

    Returns:
    float: Значение F1-меры, показывающее гармоническое среднее между точностью и полнотой.
    """
    # Ваш код здесь

In [ ]:
!pip install evaluate accelerate -U -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.0 MB/s eta 0:00:00


Если не хочется реализовывать подсчет метрик самостоятельно, можно воспользоваться библиотекой evaluate.

In [ ]:
import evaluate

metric = evaluate.load("squad")

In [ ]:
from tqdm.auto import tqdm
import collections
import numpy as np

def compute_metrics(start_logits, end_logits, features, examples, n_best, max_answer_length):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Итерируемся по всем ответам, ассоциированным с этим примером
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Пропускаем ответы, которые не полностью соответствуют контексту
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Пропускайте ответы, длина которых либо < 0, либо > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Выбираем ответ с лучшей оценкой
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

Загружаем модель и начинаем дообучение!

In [ ]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Будем использовать встроенный класс Trainer для удобства.

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-squad",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)
trainer.train()

Step,Training Loss
500,1.271000
1000,1.335500
1500,1.206700
2000,1.039500
2500,1.012500
3000,0.948900
3500,0.872500
4000,0.880300


TrainOutput(global_step=4161, training_loss=1.0628166537936907, metrics={'train_runtime': 2342.8959, 'train_samples_per_second': 113.615, 'train_steps_per_second': 1.776, 'total_flos': 2.608361755366349e+16, 'train_loss': 1.0628166537936907, 'epoch': 3.0})

Когда обучение завершено, мы можем наконец оценить нашу модель. Метод predict() класса Trainer вернет кортеж, где первыми элементами будут предсказания модели (здесь пара с начальным и конечным логитами). Мы отправляем его в нашу функцию compute_metrics():

In [ ]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"], n_best=20, max_answer_length=100)

  0%|          | 0/10570 [00:00<?, ?it/s]

{'exact_match': 74.95742667928099, 'f1': 83.57004998757499}

Отлично! Получились хорошие значения метрик. Попробуйте теперь получить exact_match больше 85% и f1 больше 90%. Используйте разные модели, разные гиперпараметры и тд.

In [ ]:
# Ваш код здесь

Проверьте на любом примере, что написанные вами метрики совпадают по значению с метриками из evaluate.

In [ ]:
# Ваш код здесь